<a href="https://colab.research.google.com/github/Kell1000/Big_data/blob/main/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %% [markdown]
## Installation de PySpark et dépendances

!pip install -q pyspark==3.5.0 pandas matplotlib seaborn nltk textblob openpyxl

# Téléchargement des ressources NLP
import nltk
nltk.download('punkt_tab', quiet=True)
nltk.download('averaged_perceptron_tagger_eng', quiet=True)
print("Dépendances installées")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 19.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 1.0.1 requires pyspark[connect]~=4.0.0, but you have pyspark 3.5.0 which is incompatible.
Dépendances installées


In [4]:
# %% [markdown]
# # Méthode 1 : Téléchargement officiel via Kaggle API
# ÉTAPE 1 : Installer kaggle
!pip install -q kaggle

# ÉTAPE 2 : Télécharger votre token Kaggle
print(""" ACTION REQUISE (1 minute) :
1. Créez un compte gratuit sur https://kaggle.com
2. Allez dans "Account Settings" → "API" → "Create New API Token"
3. Un fichier kaggle.json sera téléchargé sur votre PC
4. Exécutez la cellule ci-dessous et uploadsez ce fichier quand demandé
""")

from google.colab import files
uploaded = files.upload()  # Upload kaggle.json ici

# Déplacer le token dans le dossier de configuration
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# ÉTAPE 3 : Télécharger le dataset
print("\n Téléchargement du dataset Amazon Fine Food Reviews...")
!kaggle datasets download -d snap/amazon-fine-food-reviews -p /tmp

# Décompresser
!unzip -q /tmp/amazon-fine-food-reviews.zip -d /tmp
!ls -lh /tmp/Reviews.csv

print(f"\n Dataset téléchargé avec succès : {os.path.getsize('/tmp/Reviews.csv')/1e6:.2f} Mo")
print(f"   Nombre de lignes : {sum(1 for _ in open('/tmp/Reviews.csv')):,}")

 ACTION REQUISE (1 minute) :
1. Créez un compte gratuit sur https://kaggle.com
2. Allez dans "Account Settings" → "API" → "Create New API Token"
3. Un fichier kaggle.json sera téléchargé sur votre PC
4. Exécutez la cellule ci-dessous et uploadsez ce fichier quand demandé



Saving kaggle.json to kaggle.json

 Téléchargement du dataset Amazon Fine Food Reviews...
Dataset URL: https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews
License(s): CC0-1.0
 61% 147M/242M [00:00<00:00, 1.54GB/s]
100% 242M/242M [00:00<00:00, 971MB/s] 
replace /tmp/Reviews.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
-rw-r--r-- 1 root root 287M Sep 19  2019 /tmp/Reviews.csv

 Dataset téléchargé avec succès : 300.90 Mo
   Nombre de lignes : 568,455


In [5]:
# %% [markdown]
# Initialisation de Spark (mode local)
# Configuration adaptée aux ressources Colab (12GB RAM standard)

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Arrêt session existante si nécessaire
try:
    spark.stop()
except:
    pass

# Création session optimisée pour Colab
spark = SparkSession.builder \
    .appName("Analyse_Avis_Amazon_Colab") \
    .config("spark.driver.memory", "10g") \
    .config("spark.sql.shuffle.partitions", "8") \
    .config("spark.default.parallelism", "8") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.ui.port", "4041") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
print("SparkSession démarrée")
print(f"UI Spark disponible : http://localhost:4041 (via ngrok si nécessaire)")

SyntaxError: unexpected character after line continuation character (ipython-input-504275196.py, line 18)

In [ ]:
from pyspark import SparkContext
try:
  sc.stop()
except:
  pass

sc = SparkContext(appName='TriLogsDistribué')
log_RDD = sc.textFile("log_data.txt")

In [ ]:
pairs_RDD = log_RDD.map(lambda line: (line, None))
for line in pairs_RDD.collect():
  print(line)
print(pairs_RDD.collect())
sorted_pairs = pairs_RDD.sortByKey()
sorted_logsb = sorted_pairs.keys()


('2023-10-27T10:15:30 [ERROR] Database connection failed', None)
('2023-10-27T08:00:01 [INFO] Server started', None)
('2023-10-27T09:30:00 [WARN] Memory usage high', None)
('2023-10-27T08:05:00 [INFO] User admin logged in', None)
('2023-10-27T12:00:00 [INFO] Scheduled maintenance task', None)
('2023-10-27T10:15:25 [WARN] Latency spike detected', None)
[('2023-10-27T10:15:30 [ERROR] Database connection failed', None), ('2023-10-27T08:00:01 [INFO] Server started', None), ('2023-10-27T09:30:00 [WARN] Memory usage high', None), ('2023-10-27T08:05:00 [INFO] User admin logged in', None), ('2023-10-27T12:00:00 [INFO] Scheduled maintenance task', None), ('2023-10-27T10:15:25 [WARN] Latency spike detected', None)]


In [ ]:
out_dir = "sorted_logs"
sorted_logsb.saveAsTextFile(out_dir)

In [ ]:
res = sorted_logsb.collect()

In [ ]:
for line in res:
  print(line)

2023-10-27T08:00:01 [INFO] Server started
2023-10-27T08:05:00 [INFO] User admin logged in
2023-10-27T09:30:00 [WARN] Memory usage high
2023-10-27T10:15:25 [WARN] Latency spike detected
2023-10-27T10:15:30 [ERROR] Database connection failed
2023-10-27T12:00:00 [INFO] Scheduled maintenance task
